In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [ ]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [ ]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [ ]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [ ]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = HierarchicalNeuralTopicModel(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                               VALID:        \
       Time   Ep   Ct     LOSS   PPL      NLL     KL   REG     LOSS   PPL   
5000     89    8  463  9162.94  1857  9141.54  21.24  0.15  9160.31  1796   
10000    82   17  360  9130.95  1808  9109.57  21.27  0.13  9148.71  1780   
15000    81   26  257  9117.10  1787  9095.83  21.12  0.12  9145.18  1774   
20000    80   35  154  9109.00  1776  9087.75  21.01  0.12  9143.37  1772   
25000    80   44   51  9103.41  1768  9082.33  20.91  0.11  9142.93  1771   
30000    80   52  515  9099.50  1762  9078.56  20.82  0.11  9142.96  1771   
35000    80   61  412  9096.81  1758  9075.89  20.74  0.11  9141.70  1769   
40000    82   70  309  9094.63  1755  9073.75  20.67  0.10  9141.14  1768   
45000    80   79  206  9092.70  1752  9071.90  20.60  0.10  9140.66  1768   
50000    78   88  103  9091.04  1750  9070.32  20.54  0.10  9140.12  1767   
55000    72   97    0  9089.55  1748  9068.91  20.49  0.10  9141.03  1768   
60000    75  105  464  9088.38  1746  9067.80  20.44  0.10  9142.01  1768   
65000    73  114  361  9087.46  1744  9066.87  20.40  0.10  9139.41  1766   
70000    74  123  258  9086.54  1743  9065.96  20.36  0.10  9140.72  1768   
75000    75  132  155  9085.67  1742  9065.12  20.33  0.10  9139.00  1766   
80000    74  141   52  9084.85  1741  9064.33  20.29  0.10  9139.90  1766   
85000    75  149  516  9084.15  1740  9063.67  20.26  0.10  9139.87  1766   
90000    70  158  413  9083.58  1739  9063.12  20.22  0.09  9139.67  1766   
95000    75  167  310  9083.07  1738  9062.63  20.19  0.09  9139.82  1766   
100000   74  176  207  9082.55  1738  9062.12  20.15  0.09  9139.97  1767   
105000   72  185  104  9082.06  1737  9061.66  20.12  0.09  9139.31  1766   
110000   76  194    1  9081.55  1736  9061.18  20.08  0.09  9139.94  1767   
115000   78  202  465  9081.14  1736  9060.80  20.05  0.09  9140.19  1766   
120000   77  211  362  9080.82  1735  9060.53  20.01  0.09  9139.92  1767   
125000   74  220  259  9080.51  1735  9060.31  19.98  0.09  9139.67  1766   
130000   77  229  156  9080.20  1734  9060.09  19.95  0.09  9139.51  1766   
135000   73  238   53  9079.87  1734  9059.85  19.92  0.09  9141.20  1768   
140000   71  246  517  9079.58  1733  9059.65  19.89  0.09  9141.29  1768   
145000   71  255  414  9079.35  1733  9059.49  19.86  0.09  9139.66  1766   
150000   72  264  311  9079.13  1732  9059.35  19.84  0.09  9140.45  1767   
155000   72  273  208  9078.90  1732  9059.19  19.81  0.09  9140.77  1767   
160000   73  282  105  9078.68  1732  9059.02  19.79  0.09  9139.90  1766   
165000   72  291    2  9078.42  1731  9058.83  19.76  0.09  9140.55  1767   
170000   71  299  466  9078.22  1731  9058.69  19.74  0.09  9140.64  1767   
175000   70  308  363  9078.06  1731  9058.59  19.72  0.09  9140.67  1767   
180000   71  317  260  9077.88  1730  9058.46  19.70  0.09  9140.70  1767   
185000   69  326  157  9077.69  1730  9058.31  19.67  0.09  9139.95  1766   
190000   73  335   54  9077.48  1730  9058.16  19.65  0.09  9140.62  1767   
195000   75  343  518  9077.30  1729  9058.03  19.63  0.09  9141.52  1768   
200000   74  352  415  9077.17  1729  9057.93  19.61  0.09  9140.18  1766   
205000   81  361  312  9077.04  1729  9057.83  19.59  0.09  9139.99  1766   
210000   75  370  209  9076.89  1729  9057.73  19.57  0.09  9140.43  1767   
215000   76  379  106  9076.74  1728  9057.61  19.55  0.09  9139.59  1766   
220000   71  388    3  9076.57  1728  9057.48  19.53  0.09  9141.70  1769   
225000   74  396  467  9076.44  1728  9057.38  19.51  0.09  9141.45  1768   
230000   81  405  364  9076.34  1728  9057.31  19.49  0.09  9140.06  1767   
235000   72  414  261  9076.23  1728  9057.22  19.47  0.09  9141.18  1768   
240000   73  423  158  9076.02  1727  9057.02  19.45  0.09  9140.27  1767   
245000   72  432   55  9075.76  1727  9056.74  19.43  0.09  9141.11  1768   

                                TEST:       SPEC:             HIER:        
            NLL     

0 R: 1.000 P: 0.152 semantic verb syntactic lexical structure verbs noun type object rules
   1 R: 0.227 P: 0.059 tree parsing dependency parser node trees grammar parse nodes rules
     11 R: 0.108 P: 0.108 training models features algorithm performance probability feature learning test method
     12 R: 0.059 P: 0.059 models neural network training embeddings layer vector representations input representation
   2 R: 0.221 P: 0.078 similarity sense semantic context method vector wordnet senses models pairs
     21 R: 0.056 P: 0.056 translation english source alignment target phrase sentences translations pairs parallel
     22 R: 0.087 P: 0.087 features classification feature sentiment classifier topic positive analysis negative training
   3 R: 0.191 P: 0.064 entity features entities training extraction relation performance patterns relations precision
     31 R: 0.079 P: 0.079 languages english pos errors morphological character chinese tags tag error
     32 R: 0.048 P: 0.048 annot

# analysis

In [34]:
def get_topic_year():
    probs_topics = []
    years = []
    for i, train_batch in train_batches:
        probs_topics_batch = sess.run(model.prob_topic, feed_dict=model.get_feed_dict(train_batch, mode='test'))
        years_batch = [instance.year for instance in train_batch]
        probs_topics += [probs_topics_batch]
        years += years_batch
    probs_topics = np.concatenate(probs_topics)
    years = np.array(years)

    topic_years = years.dot(probs_topics) / np.sum(probs_topics, 0)
    topic_year = {model.topic_idxs[i]: year for i, year in enumerate(topic_years)}
    return topic_year

In [55]:
probs_topics.shape, years.shape

((36288, 16), (36288,))

In [59]:
topic_year

{0: 2006.1405287293783,
 1: 2015.073065684627,
 2: 2006.6358226095088,
 3: 2010.2605308626469,
 4: 2011.1153424999409,
 5: 2010.2201559271937,
 11: 2010.638231762466,
 12: 2013.6412160983643,
 21: 2009.3432210176409,
 22: 2003.0377479786164,
 31: 2010.7845425693845,
 32: 2007.4959372377405,
 41: 2008.9745451250799,
 42: 2009.024079592822,
 51: 2007.963976828227,
 52: 2008.7701026307714}

In [64]:
def print_topic_sample(sess, model, topic_prob_topic=None, recur_prob_topic=None, topic_freq_tokens=None, topic_year=None, parent_idx=0, depth=0):
    if depth == 0: # print root
        assert len(topic_prob_topic) == len(recur_prob_topic) == len(topic_freq_tokens) == len(topic_year)
        freq_tokens = topic_freq_tokens[parent_idx]
        recur_topic = recur_prob_topic[parent_idx]
        prob_topic = topic_prob_topic[parent_idx]
        year = topic_year[parent_idx]
        print(parent_idx, 'Avg Year: %i' % year, ' '.join(freq_tokens))
    
    child_idxs = model.tree_idxs[parent_idx]
    depth += 1
    for child_idx in child_idxs:
        freq_tokens = topic_freq_tokens[child_idx]
        recur_topic = recur_prob_topic[child_idx]
        prob_topic = topic_prob_topic[child_idx]
        year = topic_year[child_idx]
        print('  '*depth, child_idx, 'Avg Year: %i' % year, ' '.join(freq_tokens))
        
        if child_idx in model.tree_idxs: 
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens, topic_year=topic_year, parent_idx=child_idx, depth=depth)

In [65]:
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens, topic_year=topic_year)

0 Avg Year: 2006 user dialogue knowledge utterance generation domain action state systems human
   1 Avg Year: 2015 models embeddings neural vector network training vectors representations embedding representation
     11 Avg Year: 2010 document documents question topic query sentences questions answer method similarity
     12 Avg Year: 2013 sentiment features tweets negative positive classification analysis social opinion polarity
   2 Avg Year: 2006 tree parsing dependency parser grammar node trees parse rules nodes
     21 Avg Year: 2009 event relations annotation relation discourse events argument semantic annotated temporal
     22 Avg Year: 2003 verb semantic syntactic structure noun lexical verbs rules grammar phrase
   3 Avg Year: 2010 training models algorithm probability features feature function learning performance distribution
     31 Avg Year: 2010 features feature training performance entity learning classification classifier entities class
     32 Avg Year: 2007 errors